___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" />



# <font color= paletvioletred > Laboratorio 4 - Analisis Fundamental</font>




### Integrantes: 
- <Strong> Chavez Jacobo Diana Teresa </Strong> 


- <Strong> Calderón Erape Jazmín </Strong>

#### 14 Abril 2019

___
### <Strong> <font color = skyblue> I. Motivación del Laboratorio </Strong>

___
### <Strong> <font color = skyblue> II. Objetivos específicos </Strong>

In [2]:
# -- ------------------ Importar modulos a utilizar ------------------ -- #

import pandas as pd
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
#import plotly.graph_objs as go
#import plotly as py
from datetime import datetime, date, time, timedelta
import calendar

In [3]:
# -- ------------------ Parametros para Data Frame ------------------ -- #

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [4]:
# -- ------------------ Parametros para OANDA ------------------ -- #

A1_OA_Da = 16                     # Day Align
A1_OA_Ta = "America/Mexico_City"  # Time Align

A1_OA_Ai = "101-004-2221697-001"  # Id de cuenta
A1_OA_At = "practice"             # Tipo de cuenta

A1_OA_In = "USD_MXN"              # Instrumento
A1_OA_Gn = "M5"                   # Granularidad de velas

A1_OA_Ak =  "6ff" + "6703d22fd972709992df543492afe" + "-f954a9914f9b39f72fba177a23eb243" + "b" #mi api


In [5]:
# -- ------------------ Estructuras de datos ------------------ -- #



# -- Paso 1 -- Data Frame (df1_precios) descargando todos los precios, segun fechas y granularidad
# -- -- Fecha, Apertura, Maximo, Minimo, Cierre.

# -- Paso 2 -- en Data Frame (df2_operaciones)
# -- -- Fecha, Folio (1 a n), Operacion (1 = compra, -1 = venta), Unidades (0 a n),
# -- -- Margen (0 a n), Comentario ("razon con base al indicador")
# -- -- Precio_apertura, Precio_cierre

# -- Paso 3 -- en Data Frame Cuenta (df3_cuenta)
# -- -- Fecha, Capital (Efectivo $), Flotante ($),
# -- -- Balance (Capital+Flotante), Rend_Balance_acm, Comentario ("Se abrio operacion: compra/venta",
# -- -- "Se cerro operacion: Con perdia de: / Con ganancia de: ", 

# -- Paso 4 -- Generar un vector de fechas, de F1 hasta F2

par0_cap = 100000   # Capital a utilizar $100,000 Usd
par1_mar = 1000000  # Unidades por operacion maximo 1'000,000


# -- Criterios para trading
# -- para cerrar una operacion: 1) TP o SL, 2) Regla con indicador

indice = [1, 2, 3, 4, 5]
columnas = ['A','B', 'C', 'D']
df = pd.DataFrame(index=indice, columns=columnas)
df = df.fillna(0) # -- llenar todas las celdas con 0s

In [6]:
# -- ------------------ Inicializar API de OANDA ------------------ -- #

api = API(access_token=A1_OA_Ak)

In [7]:
# -- ------------------ Función de fechas para descargar precios ------------------ -- #

from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta

def date_range(start_date, end_date, increment, period):
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result

In [14]:
# -- ------------------ vector de fechas por periodos de 15 dias ------------------ -- #

start_date = date(2017, 4, 14)
end_date = date(2019, 4, 10)
fechas = date_range(start_date, end_date, 15, 'days')

F1 = fechas[0].strftime('%Y-%m-%dT%H:%M:%S')
F2 = fechas[1].strftime('%Y-%m-%dT%H:%M:%S')

___
### <Strong> <font color = skyblue> DataFrame1 Precios peso-dólar </Strong>

In [17]:
params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
          "alignmentTimezone": A1_OA_Ta, "from": F1, "to": F2}

A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
A1_Hist = api.request(A1_Req1)

In [18]:
lis = []
for i in range(len(A1_Hist['candles'])-1):
        lis.append({'TimeStamp': A1_Hist['candles'][i]['time'],
                      'Open': A1_Hist['candles'][i]['mid']['o'],
                      'High': A1_Hist['candles'][i]['mid']['h'],
                      'Low': A1_Hist['candles'][i]['mid']['l'],
                      'Close': A1_Hist['candles'][i]['mid']['c']})

pd_hist = pd.DataFrame(lis)
pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])

In [19]:
# Muestra de precios históricos del peso dólar.

pd_hist

,TimeStamp,Open,High,Low,Close
0,2017-04-14 04:00:00+00:00,18.57634,18.57638,18.57450,18.57556
1,2017-04-14 04:05:00+00:00,18.57564,18.57619,18.57479,18.57558
2,2017-04-14 04:10:00+00:00,18.57560,18.57568,18.57540,18.57547
3,2017-04-14 04:15:00+00:00,18.57550,18.57550,18.57534,18.57538
4,2017-04-14 04:20:00+00:00,18.57536,18.57560,18.56932,18.57492
...,...,...,...,...,...
3070,2017-04-28 20:30:00+00:00,18.82808,18.82991,18.82721,18.82988
3071,2017-04-28 20:35:00+00:00,18.82985,18.83002,18.82970,18.82970
3072,2017-04-28 20:40:00+00:00,18.82972,18.82973,18.82614,18.82672
3073,2017-04-28 20:45:00+00:00,18.82666,18.82719,18.82492,18.82542


___
### <Strong> <font color = skyblue> Indicador: Consumer Price Index ex Food & Energy (MoM)</Strong>

In [20]:
indicador = pd.read_csv('https://raw.githubusercontent.com/JazminErape/Lab4_eq2_AnalisisFundamental/master/Consumer%20Price%20Index%20ex%20Food%20%26%20Energy%20(MoM).csv')
pd.set_option('display.max_rows', 25)
indicador

,DateTime,Actual,Consensus,Previous
0,4/10/19 12:30,0.1,0.2,0.1
1,3/12/19 12:30,0.1,0.2,0.2
2,2/13/19 13:30,0.2,0.2,0.2
3,1/11/19 13:30,0.2,0.2,0.2
4,12/12/18 13:30,0.2,0.2,0.2
5,11/14/18 13:30,0.2,0.2,0.1
6,10/11/18 12:30,0.1,0.2,0.1
7,9/13/18 12:30,0.1,0.2,0.2
8,8/10/18 12:30,0.2,0.2,0.2
9,7/12/18 12:30,0.2,0.2,0.2


___
### <Strong> <font color = skyblue> Escenarios </Strong>

In [21]:
escenarios = []

for i in range(len(indicador['Actual'])):
    if indicador['Actual'][i]>= indicador['Consensus'][i]>=indicador['Previous'][i]: #Escenario A
        escenarios.append('A')
    elif indicador['Actual'][i]>= indicador['Consensus'][i]<=indicador['Previous'][i]:#Escenario B
        escenarios.append('B')
    elif indicador['Actual'][i]<= indicador['Consensus'][i]>=indicador['Previous'][i]: #Escenario C
        escenarios.append('C')
    elif indicador['Actual'][i]<= indicador['Consensus'][i]<=indicador['Previous'][i]: #Escenario D
        escenarios.append('D')

In [22]:
tabla=pd.DataFrame(columns=['DateTime','Actual','Consensus', 'Previous', 'Scene'])
tabla['DateTime'] = indicador['DateTime']
tabla['Actual'] = indicador['Actual']
tabla['Consensus'] = indicador['Consensus']
tabla['Previous'] = indicador['Previous']
tabla['Scene'] = escenarios
pd.set_option('display.max_rows', 25)
tabla.set_index('DateTime')


,Actual,Consensus,Previous,Scene
DateTime,,,,
4/10/19 12:30,0.1,0.2,0.1,C
3/12/19 12:30,0.1,0.2,0.2,C
2/13/19 13:30,0.2,0.2,0.2,A
1/11/19 13:30,0.2,0.2,0.2,A
12/12/18 13:30,0.2,0.2,0.2,A
11/14/18 13:30,0.2,0.2,0.1,A
10/11/18 12:30,0.1,0.2,0.1,C
9/13/18 12:30,0.1,0.2,0.2,C
8/10/18 12:30,0.2,0.2,0.2,A


___
### <Strong> <font color = skyblue> Ventanas </Strong>

In [23]:
l_F1 = ["2019-04-10T11:40:00Z", "2019-03-12T11:40:00Z", "2019-02-13T12:40:00Z", "2019-01-11T12:40:00Z",
       "2018-12-12T12:40:00Z", "2018-11-14T12:40:00Z", "2018-10-11T11:40:00Z", "2018-09-13T11:40:00Z",
       "2018-08-10T11:40:00Z", "2018-07-12T11:40:00Z", "2018-06-12T11:40:00Z", "2018-05-10T11:40:00Z",
       "2018-04-11T11:40:00Z", "2018-03-13T11:40:00Z", "2018-02-14T11:40:00Z", "2018-01-10T12:40:00Z",
       "2017-12-13T12:40:00Z", "2017-11-15T12:40:00Z", "2017-10-13T11:40:00Z", "2017-09-14T11:40:00Z", 
       "2017-09-14T11:40:00Z", "2017-07-14T11:40:00Z", "2017-06-14T11:40:00Z", "2017-05-12T11:40:00Z",
       "2017-04-14T11:40:00Z"]
l_F2 = ["2019-04-10T13:30:00Z", "2019-03-12T13:20:00Z", "2019-02-13T14:20:00Z", "2019-01-11T14:20:00Z",
       "2018-12-12T14:20:00Z", "2018-11-14T14:20:00Z", "2018-10-11T13:20:00Z", "2018-09-13T13:20:00Z",
       "2018-08-10T13:20:00Z", "2018-07-12T13:20:00Z", "2018-06-12T13:20:00Z", "2018-05-10T13:20:00Z",
       "2018-04-11T13:20:00Z", "2018-03-13T13:20:00Z", "2018-02-14T13:20:00Z", "2018-01-10T12:40:00Z",
       "2017-12-13T14:20:00Z", "2017-11-15T14:20:00Z", "2017-10-13T13:20:00Z", "2017-09-14T13:20:00Z", 
       "2017-09-14T13:20:00Z", "2017-07-14T13:20:00Z", "2017-06-14T13:20:00Z", "2017-05-12T13:20:00Z", 
       "2017-04-14T13:20:00Z"]

In [24]:
# ---------------------------------------- Obtener precios historicos -- -------------------------------------- #

datos = []
par = []
for i in range(len(l_F1)):
   
    params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
          "alignmentTimezone": A1_OA_Ta, "from": l_F1[i], "to": l_F2[i]}
    par.append(params)
    A1_Req1 = instruments.InstrumentsCandles(instrument = A1_OA_In, params=params)
    A1_Hist = api.request(A1_Req1)
    datos.append(A1_Hist)

In [25]:
lista = []
for i in range(len(A1_Hist['candles'])-1):
        lista.append({'TimeStamp': A1_Hist['candles'][i]['time'],
                      'Open': A1_Hist['candles'][i]['mid']['o'],
                      'High': A1_Hist['candles'][i]['mid']['h'],
                      'Low': A1_Hist['candles'][i]['mid']['l'],
                      'Close': A1_Hist['candles'][i]['mid']['c']})

pd_hist = pd.DataFrame(lista)
pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])

___
### <Strong> <font color = skyblue> Cálculos </Strong>

$C1$ = $High_{max}$ - $Low_{min}$  -> Volatilidad

$C2$ = $Open_{t0}$ - $Close_{t10}$  -> Dirección

$C3$ = $Open_{t0}$ - $Low_{t10}$   -> Draw Down

$C4$ = $Open_{t0}$ - $High_{t10}$   -> Draw Up

Donde:

${max}$ = Máximo valor de la ventana de precios

${min}$ = Mínimo valor de la ventana de precios

$t_{0}$ = Tiempo de ocurrencia del indicador

$t_{10}$ = Último dato de la ventana de precios